In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from urllib.parse import urljoin
import pandas as pd
import time
import re
import os 
import re
import csv


In [2]:
URL = "https://www.ankhang.vn/chinh-sach-bao-hanh.html"
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
driver.get(URL)

In [11]:
def cleaning_text(text : str)-> str:
    """
     loai bo cac ki tu xuong dong khi chua ket thuc cau, xoa khoang trong thua.
    """
    # thay xuong dong thanh khoang trang khi chua ket thuc cau
    text = re.sub(r'(?<![.!?;:])\n(?!\n)', ' ', text)
    # xoa khoang trong thua
    text = re.sub(r'\n{2,}', '\n\n', text)
    text = re.sub(r'^\s*-{3,}\s*$', '', text, flags=re.MULTILINE)
    # 1. Xóa các khối điều kiện MSO (Microsoft Word/Outlook HTML artifacts)
    text = re.sub(r'\[if\s+gte\s+mso\s+\d+\]>.*?<!\[endif\]', '', text, flags=re.DOTALL)
    
    return text.strip()

In [ ]:
import requests
from bs4 import BeautifulSoup, NavigableString
import pandas as pd

out_put_file = ["csvc.txt", "csbh.txt","csdt.txt"]
url = ["https://www.ankhang.vn/chinh-sach-van-chuyen.html", "https://www.ankhang.vn/chinh-sach-bao-hanh.html", "https://www.ankhang.vn/chinh-sach-doi-tra.html"]
for i in range(3):
    res = requests.get(url[i])
    res.encoding = "utf-8"
    soup = BeautifulSoup(res.text, "html.parser")

    content = soup.find("div", class_="nd")
    if not content:
        raise ValueError("Không tìm thấy phần nội dung chính")

    # Dùng list để ghi lại các phần tử cần xử lý (không đệ quy sâu vào table)
    elements_to_process = list(content.children)
    markdown_output = ""

    for elem in elements_to_process:
        if not elem.name:  # là NavigableString thuần
            text = elem.strip()
            if text:
                markdown_output += cleaning_text(text) + "\n"
            continue

        # Bỏ các thẻ không cần thiết
        if elem.name in ["script", "style", "noscript"]:
            continue

        # Xử lý tiêu đề h1-h6
        if elem.name.startswith("h") and len(elem.name) == 2 and elem.name[1].isdigit():
            level = int(elem.name[1])
            text = elem.get_text(strip=True)
            if text:
                markdown_output += "#" * level + " " + cleaning_text(text) + "\n"

        # Xử lý đoạn văn
        elif elem.name == "p":
            text = elem.get_text(strip=True)
            if text:
                markdown_output += cleaning_text(text) + "\n"

        # Xử lý danh sách
        elif elem.name in ["ul", "ol"]:
            for li in elem.find_all("li", recursive=False):
                marker = "- " if elem.name == "ul" else "1. "
                text = li.get_text(strip=True)
                if text:
                    markdown_output += marker + cleaning_text(text) 
            # markdown_output += "\n"

        # Xử lý bảng: chỉ xử lý một lần, KHÔNG duyệt sâu vào td/th
        elif elem.name == "table":
            try:
                df = pd.read_html(str(elem))[0]
                md_table = df.to_markdown(index=False)
                markdown_output += md_table + "\n\n"
            except Exception as e:
                pass

     

    # Xuất kết quả
    print(markdown_output)

    # Lưu file
    output_file = out_put_file[i]
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(markdown_output)


C:\Users\ADZ\AppData\Local\Temp\ipykernel_5860\402681767.py:56: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(elem))[0]
C:\Users\ADZ\AppData\Local\Temp\ipykernel_5860\402681767.py:56: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(elem))[0]



Khi mua hàng tại An Khang Computer, Quý Khách có thể lựa chọn một trong các hình thức vận chuyển sau:
- An Khang Computer trực tiếp vận chuyển và giao hàng tận tay Khách hàng- An Khang Computer giao hàng đến Khách hàng thông qua dịch vụ vận chuyển bên thứ 3: Xe Tải hàng hóa, Xe Khách tuyến cố định; Đơn vị Logistic chuyên nghiệp…
2.1  An Khang Computer trực tiếp vận chuyển và giao hàng tận tay Khách hàng.
| 0     | 1                                       | 2                                                                  | 3                                   | 4                                                                   | 5                                                                                                       |
|:------|:----------------------------------------|:-------------------------------------------------------------------|:------------------------------------|:--------------------------------------------------------------------|:---------------------------

C:\Users\ADZ\AppData\Local\Temp\ipykernel_5860\402681767.py:56: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(elem))[0]


# CHÍNH SÁCH BẢO HÀNH
## 1. LIÊN HỆ BẢO HÀNH SẢN PHẨM
Khi có nhu cầu bảo hành sản phẩm, Quý khách hàng vui lòng liên hệ với An Khang Computer qua các hình thức sau:
- Gọi tổng đài 1900 2655 (nhánh 106).
- Gửi tin nhắn trực tiếp tại website hoặc fanpage.
- Mang sản phẩm trực tiếp đến Trung tâm bảo hành của An Khang Computer: số 25 Yên Lãng – Đống Đa – Hà Nội.
## 2. ĐIỀU KIỆN BẢO HÀNH
- Sản phẩm còn trong thời hạn bảo hành của Hãng sản xuất. Thời hạn bảo hành được ghi nhận dựa trên thông tin hóa đơn mua hàng (ngày mua, số điện thoại), số serial sản phẩm.
- Sản phẩm phải còn tem niêm phong bảo hành hoặc tem của nhà phân phối. Với các sản phẩm cần bảo hành theo hộp, khách hàng phải gửi đầy đủ hộp và phụ kiện đi kèm.
- Sản phẩm còn nguyên trạng. Không trầy xước, cấn móp, cong vênh, biến dạng, cháy nổ, ẩm mốc... ngoài quy định của hãng/ nhà phân phối.
- Sản phẩm phát sinh lỗi trong quá trình sử dụng do nhà sản xuất như linh kiện, lỗi kỹ thuật.
Lưu ý:
- Các điều kiện bảo hành cụ thể có thể th

C:\Users\ADZ\AppData\Local\Temp\ipykernel_5860\402681767.py:56: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(elem))[0]
